## Análise e Geração de Gráficos

In [2]:
!pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

#### Import e Tratamento dos dados da otimização de hiperparâmetros 

In [4]:
results = pd.read_csv("results.csv")

In [5]:
def convert_to_array(s):
    s = s.replace('\n', '')  # Remove \n
    s = s.replace('[', '').replace(']', '')  # Remove colchetes
    array = np.array([float(x) for x in s.split()])  # Converte para array NumPy
    return array

In [6]:
results['Solução Predita'] = results['Solução Predita'].apply(convert_to_array)

In [7]:
results.head()

,ID,Taxa de Aprendizado,Número de Camadas Ocultas,Número de Neurônios por Camada,Ativação,Solução Predita,Loss
0,0,0.0010,5,60,tanh,"[-0.00896674, -0.0086022, -0.0083431, -0.00819...",0.010614
1,1,0.0050,6,40,ReLU,"[0.03829777, 0.03835155, 0.03837201, 0.0383924...",0.252706
2,2,0.0050,6,70,Swish,"[-0.000918082893, -0.000892743468, -0.00086864...",0.000126
3,3,0.0005,7,60,tanh,"[0.00639288547, 0.005788852, 0.0051537645, 0.0...",0.002465
4,4,0.0010,4,80,sin,"[0.00178459939, 0.00194640364, 0.00210815575, ...",0.000995


### Gráfico Comparativo usando a seção x = 5, t = 2 * pi

In [22]:
# Ordenar os resultados pela 'Loss'
sorted_results = results.sort_values(by='Loss')

# Pega o top 10
top_10_results = sorted_results.head(10)

In [14]:
# Configurações do Seaborn
palette = sns.color_palette("rocket")
sns.set_palette(palette)
sns.set_style("ticks", {'axes.grid': True})

# Criar a figura e o eixo
fig, ax1 = plt.subplots(figsize=(16, 8))

# Definir os rótulos dos eixos com LaTeX
ax1.set_xlabel(r'$y$', fontsize=14)
ax1.set_ylabel(r'$u(x = 5, y, t = 2\pi)$', fontsize=14)

# Número de valores
num_values = 128
y = np.linspace(0, 10, num_values).reshape(-1, 1)

# Plotar as linhas com Seaborn
for i, (_, result) in enumerate(top_10_results.iterrows()):
    label = f"{result['ID']} - LR: {result['Taxa de Aprendizado']}, Layers: {result['Número de Camadas Ocultas']}, Nodes: {result['Número de Neurônios por Camada']}, Activ.: {result['Ativação']}"
    sns.lineplot(x=y.flatten(), y=result['Solução Predita'], linewidth=2, label=label, ax=ax1)

# Adicionar a legenda
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Salvar a figura
fig.savefig('plots/comparison_plot.png', bbox_inches='tight')
plt.close(fig)

### Gráfico comparativo log(loss) x sort(m)

In [15]:
# Criação do gráfico de pontos
fig, ax = plt.subplots()
sns.lineplot(x=np.arange(0, 40, 1), y=np.log10(sorted_results['Loss']), marker='o', ax=ax)

# Configuração dos eixos e título
ax.set_xlabel('Ranking')
ax.set_ylabel(r'$log_{10}(\mathcal{L}_{\theta^{*}}[\lambda]$)')

# Ajustes adicionais para melhorar a visualização em escala logarítmica
ax.grid(True, which="both", ls="--")
ax.minorticks_on()

# Salvar o gráfico
fig.savefig('plots/loss_hpo.png', bbox_inches='tight')
plt.close(fig)

### Gráfico comparando melhor solução contra solver numérico

In [9]:
sol_malha_128 = pd.read_csv("sol_malha_128x128.csv")

In [10]:
sol_malha_128.head()

,u,vtkValidPointMask,arc_length,Points:0,Points:1,Points:2
0,0.000000e+00,1,0.000000,5,0.000000,0
1,-1.874100e-09,1,0.078125,5,0.078125,0
2,-1.980800e-11,1,0.156250,5,0.156250,0
3,8.355700e-10,1,0.234380,5,0.234380,0
4,-1.712200e-12,1,0.312500,5,0.312500,0


In [26]:
# Criar a figura e o eixo
fig, ax1 = plt.subplots(figsize=(16, 8))

# Definir os rótulos dos eixos com LaTeX
ax1.set_xlabel(r'$y$', fontsize=14)
ax1.set_ylabel(r'$u(x = 5, y, t = 2\pi)$', fontsize=14)

#Plot do melhor modelo
label = f"{sorted_results.iloc[0]['ID']} - LR: {sorted_results.iloc[0]['Taxa de Aprendizado']}, Layers: {sorted_results.iloc[0]['Número de Camadas Ocultas']}, Nodes: {sorted_results.iloc[0]['Número de Neurônios por Camada']}, Activ.: {sorted_results.iloc[0]['Ativação']}"
sns.lineplot(x=y.flatten(), y=sorted_results.iloc[0]['Solução Predita'], linewidth=2, label=label, ax=ax1)

label = f"Solver Numérico"
sns.lineplot(x=y.flatten(), y=sol_malha_128.iloc[:-1]["u"], linewidth=2, label=label, ax=ax1)

# Salvar a figura
fig.savefig('plots/best_compare_solver.png', bbox_inches='tight')
plt.close(fig)